In [21]:
jai bhuriya baba ji/om nmay shivay

SyntaxError: invalid syntax (<ipython-input-21-9a4e91911a60>, line 1)

In [22]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import sqlite3

import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn import cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


import sys, os
import matplotlib.pyplot as plt

In [23]:
data=sqlite3.connect("database.sqlite")

In [24]:
import pandas as pd
filtered_data=pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score !=3
""",data)
def partition(x):
    if x<3:
        return 'negative'
    return 'positive'
a_score=filtered_data["Score"]
positiveNegative=a_score.map(partition)
filtered_data["Score"]=positiveNegative

In [25]:
print(filtered_data.shape)

(525814, 10)


In [26]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [27]:
sorted_data=filtered_data.sort_values('ProductId',ascending=True,inplace=False,kind='quickSort',na_position='last')
#de duplication of entries
final=sorted_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'},keep='first',inplace=False)
final.shape

(364173, 10)

In [28]:
final=final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]
print(final.shape)
#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(364171, 10)


positive    307061
negative     57110
Name: Score, dtype: int64

In [29]:
%%time
import re
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stop=set(stopwords.words('english'))
print(stop)
sno=nltk.stem.SnowballStemmer('english')
print(sno.stem('tasty'))

def cleanhtml(sentence):#function to clean the word of any html-tags
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr,'',sentence)
    return cleantext
def cleanpunc(sentence):#function to clean the word of any punctuation or special characters
    cleaned=re.sub(r'[!|?|<|,|/|.]',r'',sentence)
    cleaned=re.sub(r'[;|"|\|)]',r'',cleaned)
    return cleaned

{'yourself', 'mightn', 'for', 'are', 'hadn', "hadn't", 'themselves', "won't", 'into', 'his', "didn't", 'll', 'having', 'until', 'on', 'that', 'am', 'when', 'again', 'up', 'isn', 'down', 'because', 'and', "it's", "hasn't", 'ourselves', 'these', 'those', 'itself', 'once', 'some', 'do', 'a', 'above', "you've", 'only', 'they', 'myself', 'being', 'doing', 'their', 'further', 'at', "mightn't", 'm', 'its', 'been', 'by', 'why', 'just', 'here', 'ma', "wouldn't", 'what', 'we', 'she', 'theirs', 'who', 'so', 'while', 'with', 't', "mustn't", 'after', 'about', 'had', 'if', 'did', "isn't", 'the', "you'll", 'or', 'than', 'during', 'which', 'between', 'does', "shouldn't", 'over', 'mustn', 'will', 'shouldn', 'our', 'is', 'an', 'below', 'have', 'was', "should've", 'haven', 'hasn', "needn't", 'herself', "don't", 's', 'don', 'shan', "couldn't", 'were', 'from', "wasn't", 'me', "doesn't", 'be', 'other', 'needn', 'ain', 'whom', 'has', 'your', 'my', 'you', "weren't", 've', 'any', 'didn', 'not', 'as', 'of', 'ca

In [30]:
%%time

#Code for implementing step-by-step the checks mentioned in the pre-processing phase
j=0
str1=''
sn=''
final_string=[]
positive_R=[]
negative_R=[]
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent)
    sent=cleanhtml(sent)#remove html tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha())& (len(cleaned_words)>2):
                if(cleaned_words.lower() not in stop):
                    sn=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(sn)
                    if(final['Score'].values)[j]=='positive':
                        positive_R.append(sn)
                    if(final['Score'].values)[j]=='negative':
                        negative_R.append(sn)
                        
                else:
                    continue
            else:
                continue
       #print(filtered_sentance)
    str1=b" ".join(filtered_sentence) #final string of clean_words
    #print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    final_string.append(str1)
    j+=1


CPU times: user 6min 8s, sys: 415 ms, total: 6min 9s
Wall time: 6min 9s


In [ ]:
%%time

final['cleaned_Text']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review
final['cleaned_Text']=final['cleaned_Text'].str.decode('utf-8')
print(final.head(3))


print(final.columns)

In [32]:
%%time
from datetime import timedelta
#sorting based on time
#final['Time']=pd.to_datetime(final['Time'])
final['Time']=pd.to_datetime(final.Time)

final=final.sort_values('Time',ascending=True,inplace=False,kind='quickSort',na_position='last')

CPU times: user 2.44 s, sys: 40 ms, total: 2.48 s
Wall time: 849 ms


In [33]:
final.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,cleaned_Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,1970-01-01 00:00:00.939340800,EVERY book is educational,this witty little book makes my son laugh at l...,witti littl book make son laugh loud recit car...
138683,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,positive,1970-01-01 00:00:00.940809600,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,rememb see show air televis year ago child sis...


In [34]:
X_train, y_train = final['cleaned_Text'].iloc[0:80000], final['Score'].iloc[0:80000]
#X_cv, y_cv = final['cleaned_Text'].iloc[60000:80000], final['Score'].iloc[60000:80000]
X_test, y_test = final['cleaned_Text'].iloc[80000:100000], final['Score'].iloc[80000:100000]


# 1.BOW

In [35]:
#BoW
count_vect = CountVectorizer()
count_vect.fit(X_train)
X_train=count_vect.transform(X_train)
#X_cv=count_vect.transform(X_cv)
X_test=count_vect.transform(X_test)
print(X_train.shape,X_test.shape)

(80000, 67317) (20000, 67317)


In [36]:
#Standerdized the data
scaler = StandardScaler(with_mean=False)
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

<20000x67317 sparse matrix of type '<class 'numpy.float64'>'
	with 580348 stored elements in Compressed Sparse Row format>

In [ ]:
%%time
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import *
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import TimeSeriesSplit

tuned_parameters = [{'C': [10**-4, 10**-2, 10**0, 10**2, 10**4]}]
#Using GridSearchCV

# Create a linear SVM classifier 

model = GridSearchCV(svm.SVC(kernel='linear',class_weight='balanced'), tuned_parameters, scoring = 'accuracy', cv=5)
model.fit(X_train,y_train)


print(model.best_estimator_)



print("best_score")
print(model.best_score_)



print('Parameters')
print(model.best_params_)



print(model.score(X_test,y_test))


In [ ]:
clf = svm.SVC(kernel='linear', C=1...,class_weight='balanced')

In [ ]:
# Train classifier 
clf.fit(X_train, y_train)
 

In [ ]:
# Plot decision function on training and test data
plot_decision_function(X_train, y_train, X_test, y_test, clf)

In [ ]:
# Make predictions on unseen test data
clf_predictions = clf.predict(X_test)
print("Accuracy: {}%".format(clf.score(X_test, y_test) * 100 ))

In [ ]:
#cv = CountVectorizer()
#cv.fit(data)
print len(count_vect.vocabulary_)
print count_vect.get_feature_names()



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt

def plot_coefficients(classifier, feature_names, top_features=20):
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
 # create plot
    plt.figure(figsize=(15, 5))
    colors = [‘red’ if c < 0 else ‘blue’ for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(np.arange(1, 1 + 2 * top_features), feature_names[top_coefficients], rotation=60, ha=’right’)
    plt.show()

#cv = CountVectorizer()
#cv.fit(data)
#print len(cv.vocabulary_)
#print cv.get_feature_names()
#X_train = cv.transform(data)

#svm = LinearSVC()
#svm.fit(X_train, y_train)

plot_coefficients(clf, count_vect.get_feature_names())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt

def plot_coefficients(classifier, feature_names, top_features=20):
 coef = classifier.coef_.ravel()
 top_positive_coefficients = np.argsort(coef)[-top_features:]
 top_negative_coefficients = np.argsort(coef)[:top_features]
 top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
 # create plot
 plt.figure(figsize=(15, 5))
 colors = [‘red’ if c < 0 else ‘blue’ for c in coef[top_coefficients]]
 plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
 feature_names = np.array(feature_names)
 plt.xticks(np.arange(1, 1 + 2 * top_features), feature_names[top_coefficients], rotation=60, ha=’right’)
 plt.show()

cv = CountVectorizer()
cv.fit(data)
print len(cv.vocabulary_)
print cv.get_feature_names()

X_train = cv.transform(data)

svm = LinearSVC()
svm.fit(X_train, target)

plot_coefficients(svm, cv.get_feature_names())

# Random Search Linear SVM

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn import svm
from scipy.stats import uniform
#tuned_parameter = [{'C': [10**-3, 10**-2, 10**0, 10**2, 10**3]}]
tuned_parameter = {'C':uniform()}
model1 = RandomizedSearchCV(svm.SVC(kernel='linear',class_weight='balanced'),tuned_parameters, scoring = 'accuracy', cv=5)
model1.fit(X_train,y_train)


In [ ]:
print(model1.best_estimator_)

In [ ]:
print("best_score")
print(model1.best_score_)

In [ ]:
print('Parameters')
print(model1.best_params_)

In [ ]:
print(model1.score(X_test,y_test))


In [ ]:
clf = svm.SVC(kernel='linear', C=1..............,class_weight='balanced')

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
# Plot decision function on training and test data
plot_decision_function(X_train, y_train, X_test, y_test, clf)

In [ ]:
# Make predictions on unseen test data
clf_predictions = clf.predict(X_test)
print("Accuracy: {}%".format(clf.score(X_test, y_test) * 100 ))

# SVM-RBF-GridsearchCv

In [ ]:
from sklearn import svm
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=300)
X_train = svd.fit_transform(X_train)
X_test = svd.fit_transform(X_test)






tuned_parameters = [{'C': [10**-4, 10**-2, 10**0, 10**2, 10**4],'gamma':[10**-4, 10**-2, 10**0, 10**2, 10**4]}]
#Using GridSearchCV

# Create a linear SVM classifier 


model = GridSearchCV(svm.SVC(kernel= 'rbf',class_weight='balanced'), tuned_parameters, scoring = 'accuracy', cv=5)

model.fit(X_train,y_train)

print(model.best_estimator_)


print("best_score")
print(model.best_score_)


print('Parameters')
print(model.best_params_)




#model_mean_scores = [result.mean_validation_score for result in model.grid_scores_]
print(model.score(X_test,y_test))


In [ ]:
# Create SVM classifier based on RBF kernel. 
clf = svm.SVC(kernel='rbf', C = 10.0...................., gamma=0.1................................,class_weight='balanced')

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
clf.predict(X_test)

In [ ]:
# Plot decision function on training and test data
plot_decision_function(X_train, y_train, X_test, y_test, clf)

In [ ]:
print("Accuracy: {}%".format(clf.score(X_test, y_test) * 100 ))

# Random SearchCV-RBF

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn import svm

from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=300)
X_train = svd.fit_transform(X_train)
X_test = svd.fit_transform(X_test)



tuned_parameter = {'C':uniform()}
model2 = RandomizedSearchCV(svm.SVC(kernel='rbf',class_weight='balanced'),tuned_parameters, scoring = 'accuracy', cv=5)
model2.fit(X_train,y_train)


In [ ]:
print(model2.best_estimator_)

In [ ]:
print("best_score")
print(model2.best_score_)

In [ ]:
print('Parameters')
print(model2.best_params_)

In [ ]:
print(model2.score(X_test,y_test))


In [ ]:
# Create SVM classifier based on RBF kernel. 
clf = svm.SVC(kernel='rbf', C = 10.0...................., gamma=0.1................................,class_weight='balanced')

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
clf.predict(X_test)

In [ ]:
# Plot decision function on training and test data
plot_decision_function(X_train, y_train, X_test, y_test, clf)

In [ ]:
print("Accuracy: {}%".format(clf.score(X_test, y_test) * 100 ))

# 2.TF-IDF

In [ ]:
X_train, y_train = final['cleaned_Text'].iloc[0:80000], final['Score'].iloc[0:80000]
#X_cv, y_cv = final['cleaned_Text'].iloc[60000:80000], final['Score'].iloc[60000:80000]
X_test, y_test = final['cleaned_Text'].iloc[80000:100000], final['Score'].iloc[80000:100000]



#Initializing tfidf vectorizer
tfidf_vect = TfidfVectorizer()

#Fitting for tfidf vectorization
tfidf_vect.fit(X_train)
X_train= tfidf_vect.transform(X_train)
X_test = tfidf_vect.transform(X_test)
#X_cv = tfidf_vect.transform(X_cv)
X_train.shape,y_train.shape

In [ ]:
#Standerdized the data
scaler = StandardScaler(with_mean=False)
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

# GridSearchCV-LinearSVM

In [ ]:
%%time
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import *
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import TimeSeriesSplit


tuned_parameters = [{'C': [10**-4, 10**-2, 10**0, 10**2, 10**4]}]
#Using GridSearchCV

# Create a linear SVM classifier 

model = GridSearchCV(svm.SVC(kernel='linear',class_weight='balanced'), tuned_parameters, scoring = 'accuracy', cv=5)
model.fit(X_train,y_train)

print(model.best_estimator_)


print("best_score")
print(model.best_score_)


print('Parameters')
print(model.best_params_)


print(model.score(X_test,y_test))

In [ ]:
clf = svm.SVC(kernel='linear', C=1..............,class_weight='balanced')

In [ ]:
# Train classifier 
clf.fit(X_train, y_train)
 

In [ ]:
# Plot decision function on training and test data
plot_decision_function(X_train, y_train, X_test, y_test, clf)

In [ ]:
# Make predictions on unseen test data
clf_predictions = clf.predict(X_test)
print("Accuracy: {}%".format(clf.score(X_test, y_test) * 100 ))

 #  Random Search Linear SVM

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn import svm
#tuned_parameter = [{'C': [10**-3, 10**-2, 10**0, 10**2, 10**3]}]
tuned_parameter = {'C':uniform()}
model1 = RandomizedSearchCV(svm.SVC(kernel='linear',class_weight='balanced'),tuned_parameters, scoring = 'accuracy', cv=5)
model1.fit(X_train,y_train)

In [ ]:
print(model1.best_estimator_)

In [ ]:
print("best_score")
print(model1.best_score_)

In [ ]:
print('Parameters')
print(model1.best_params_)

In [ ]:
print(model1.score(X_test,y_test))


In [ ]:
clf = svm.SVC(kernel='linear', C=1..............,class_weight='balanced')

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
# Plot decision function on training and test data
plot_decision_function(X_train, y_train, X_test, y_test, clf)

In [ ]:
# Make predictions on unseen test data
clf_predictions = clf.predict(X_test)
print("Accuracy: {}%".format(clf.score(X_test, y_test) * 100 ))

 # SVM-RBF-Gridsearch-Tfidf

In [ ]:
from sklearn import svm
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=300)
X_train = svd.fit_transform(X_train)
X_test = svd.fit_transform(X_test)

tuned_parameters = [{'C': [10**-4, 10**-2, 10**0, 10**2, 10**4],'gamma':[10**-4, 10**-2, 10**0, 10**2, 10**4]}]
#Using GridSearchCV

# Create a linear SVM classifier 


model = GridSearchCV(svm.SVC(kernel= 'rbf',class_weight='balanced'), tuned_parameters, scoring = 'accuracy', cv=5)

model.fit(X_train,y_train)


In [ ]:
print(model.best_estimator_)

In [ ]:
print("best_score")
print(model.best_score_)

In [ ]:
print('Parameters')
print(model.best_params_)

In [ ]:

print(model.score(X_test,y_test))

In [ ]:
# Create SVM classifier based on RBF kernel. 
clf = svm.SVC(kernel='rbf', C = 10.0...................., gamma=0.1................................,class_weight='balanced')

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
clf.predict(X_test)

In [ ]:
# Plot decision function on training and test data
plot_decision_function(X_train, y_train, X_test, y_test, clf)

In [ ]:
print("Accuracy: {}%".format(clf.score(X_test, y_test) * 100 ))

# RandomeSearchCV-RBF-Tfidf

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn import svm
#tuned_parameter = [{'C': [10**-3, 10**-2, 10**0, 10**2, 10**3]}]
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=300)
X_train = svd.fit_transform(X_train)
X_test = svd.fit_transform(X_test)



tuned_parameter = {'C':uniform()}
model1 = RandomizedSearchCV(svm.SVC(kernel='rbf',class_weight='balanced'),tuned_parameters, scoring = 'accuracy', cv=5)
model1.fit(X_train,y_train)


In [ ]:
print(model.best_estimator_)

In [ ]:
print("best_score")
print(model.best_score_)

In [ ]:
print('Parameters')
print(model.best_params_)

In [ ]:
print(model.score(X_test,y_test))

In [ ]:
# Create SVM classifier based on RBF kernel. 
clf = svm.SVC(kernel='rbf', C = 10.0...................., gamma=0.1................................,class_weight='balanced')

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
clf.predict(X_test)

In [ ]:
# Plot decision function on training and test data
plot_decision_function(X_train, y_train, X_test, y_test, clf)

In [ ]:
print("Accuracy: {}%".format(clf.score(X_test, y_test) * 100 ))

# 3.WORD2Vec

In [ ]:
import re

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [ ]:
#Train your own Word2Vec model using your own train text corpus
#spliting the data
X_train, y_train = final['cleaned_Text'].iloc[0:80000],final['Score'].iloc[0:80000]
#X_cv, y_cv = final['cleaned_Text'].iloc[60000:80000],final['Score'].iloc[60000:80000]
X_test, y_test = final['cleaned_Text'].iloc[80000:100000],final['Score'].iloc[80000:100000]
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
    

In [ ]:
import gensim
from gensim import utils
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
from gensim import parsing, corpora, matutils, interfaces, models, similarities, summarization, utils


train_sent=[]

for sent in X_train:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    train_sent.append(filtered_sentence)
print(len(train_sent))

In [ ]:
test_sent=[]

for sent in X_test:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    test_sent.append(filtered_sentence)
print(len(test_sent))

In [ ]:
train_w2v_model = gensim.models.Word2Vec(train_sent,min_count=5,size=50,workers=4)
train_words = list(train_w2v_model.wv.vocab)
print(len(train_words))

# 3.1 AvgW2v

In [ ]:
#avg w2v
#Avg-w2v for train data

train_vectors = []
for sent in tqdm(train_sent):
    train_vec = np.zeros(50)
    cnt_words = 0
    for word in sent:
        try:
            vec=train_w2v_model.wv[word]
            train_vec+=vec
            cnt_words+=1
        except:
            pass
    train_vec/=cnt_words
    train_vectors.append(train_vec)
print(len(train_vectors))
print(len(train_vectors[0]))


In [ ]:
%%time
#Avg-w2v for test data
test_vectors = []
for sent in tqdm(test_sent):
    test_vec = np.zeros(50)
    cnt_words = 0
    for word in sent:
        try:
            vec=train_w2v_model.wv[word]
            test_vec+=vec
            cnt_words+=1
        except:
            pass
    test_vec/=cnt_words
    test_vectors.append(test_vec)
print(len(test_vectors))
print(len(test_vectors[0]))


In [ ]:
X_train = train_vectors

X_test = test_vectors
print(len(X_train),len(X_test))

 # TFIDF-W2v

In [ ]:
#TFIDF-W2V


X_train, y_train = final['cleaned_Text'].iloc[0:60000], final['Score'].iloc[0:60000]
X_cv, y_cv = final['cleaned_Text'].iloc[60000:80000], final['Score'].iloc[60000:80000]
X_test, y_test = final['cleaned_Text'].iloc[80000:100000], final['Score'].iloc[80000:100000]

# Initializing tfidf vectorizer
tfidf_vect = TfidfVectorizer(ngram_range=(1, 1))

# Fitting for tfidf vectorization
tfidf_vect.fit(X_train)
X_train = tfidf_vect.transform(X_train)
X_test = tfidf_vect.transform(X_test)
X_cv = tfidf_vect.transform(X_cv)
#we are converting dictionary with word as key and idf as value
dictionary = dict(zip(tfidf_vect.get_feature_names(),list(tfidf_vect.idf_)))
print(X_train.shape,y_train.shape)


#tfidf_feat=tfidf_vect.get_feature_names()# tfidf word/col names

In [ ]:
%%time
tfidf_feat = tfidf_vect.get_feature_names()
tfidf_train_vectors = []
row = 0
for sent in tqdm(train_sent):
    sent_vec = np.zeros(50)
    weight_sum = 0
    for word in sent:
        try:
            vec = train_w2v_model.wv[word]
            #tf_idf = X_train[row, tfidf_feat.index(word)](prev.............)
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
        except:
            pass
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_train_vectors.append(sent_vec)
    row += 1
    

In [ ]:
len(tfidf_train_vectors)

In [ ]:
%%time
#tfidf w2v for cv data
tfidf_feat = tfidf_vect.get_feature_names()
tfidf_cv_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in cv_sent: # for each review/sentence
    cv_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = train_w2v_model.wv[word]
            # obtain the tf_idf of a word in a sentence/review
            #tf_idf = X_cv[row, tfidf_feat.index(word)]
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            cv_vec += (vec * tf_idf)
            weight_sum += tf_idf
        except:
            pass
    cv_vec /= weight_sum
    tfidf_cv_vectors.append(cv_vec)
    row += 1
len(tfidf_cv_vectors)

In [ ]:
#tfidf w2v for test data

tfidf_test_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in test_sent: # for each review/sentence
    test_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = train_w2v_model.wv[word]
            # obtain the tf_idf of a word in a sentence/review
            #tf_idf = X_test[row, tfidf_feat.index(word)]
            tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            test_vec += (vec * tf_idf)
            weight_sum += tf_idf
        except:
            pass
    test_vec /= weight_sum
    tfidf_test_vectors.append(test_vec)
    row += 1
len(tfidf_test_vectors)

In [ ]:
X_train = tfidf_train_vectors
X_cv = tfidf_cv_vectors
X_test = tfidf_test_vectors 
len(X_train)
print(len(X_cv))
print(len(y_cv))